In [1]:
import os
from dotenv import load_dotenv
from typing import Any

from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIResponsesClient

from agent_framework import Message, WorkflowEvent
from agent_framework.orchestrations import ConcurrentBuilder

In [2]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

Project Endpoint:  https://practicefoundryv10.services.ai.azure.com/api/projects/practicefoundryv10project
Model:  gpt-4o


In [3]:
credential = AzureCliCredential()
chat_client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

In [5]:
from agent_framework import Agent

researcher = Agent(
    name="Researcher",
    description="Collects relevant background information.",
    instructions="Gather concise facts that help answer the question. Be brief and factual.",
    client=chat_client,
)

writer = Agent(
    name="Writer",
    description="Synthesizes polished answers using gathered information.",
    instructions="Compose clear, structured answers using any notes provided. Be comprehensive.",
    client=chat_client,
)

orchestrator_agent = Agent(
    name="Orchestrator",
    description="Coordinates multi-agent collaboration by selecting speakers",
    instructions="""
You coordinate a team conversation to solve the user's task.

Guidelines:
- Start with Researcher to gather information
- Then have Writer synthesize the final answer
- Only finish after both have contributed meaningfully
""",
    client=chat_client,
)

In [6]:
from agent_framework.orchestrations import GroupChatBuilder, GroupChatState

def round_robin_selector(state: GroupChatState) -> str:
    """A round-robin selector function that picks the next speaker based on the current round index."""

    participant_names = list(state.participants.keys())
    return participant_names[state.current_round % len(participant_names)]


# Build the group chat workflow
workflow = GroupChatBuilder(
    participants=[researcher, writer],
    termination_condition=lambda conversation: len(conversation) >= 4,
    selection_func=round_robin_selector,
).build()

In [8]:
from typing import cast
from agent_framework import AgentResponseUpdate, Role

task = "What are the key benefits of async/await in Python?"

print(f"Task: {task}\n")
print("=" * 80)

final_conversation: list[Message] = []
last_executor_id: str | None = None

# Run the workflow
async for event in workflow.run(task, stream=True):
    if event.type == "output" and isinstance(event.data, AgentResponseUpdate):
        # Print streaming agent updates
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"[{eid}]:", end=" ", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif event.type == "output":
        # Workflow completed - data is a list of Message
        final_conversation = cast(list[Message], event.data)

if final_conversation:
    print("\n\n" + "=" * 80)
    print("Final Conversation:")
    for msg in final_conversation:
        author = getattr(msg, "author_name", "Unknown")
        text = getattr(msg, "text", str(msg))
        print(f"\n[{author}]\n{text}")
        print("-" * 80)

print("\nWorkflow completed.")

Task: What are the key benefits of async/await in Python?



Final Conversation:

[None]
What are the key benefits of async/await in Python?
--------------------------------------------------------------------------------

[Researcher]
1. **Improved Readability**: Async/await syntax resembles synchronous code, making it easier to read and understand.

2. **Efficient I/O Operations**: It allows for non-blocking I/O operations, enabling other tasks to run while waiting for I/O-bound tasks to complete.

3. **Simplified Error Handling**: Errors in asynchronous functions can be handled similarly to synchronous code, using try/except blocks.

4. **Better Resource Utilization**: It allows concurrent execution without the overhead of threading, improving resource use.

5. **Easier Debugging**: The clear structure helps with debugging code by maintaining a straightforward call stack.

6. **Avoid Callback Hell**: Async/await eliminates deeply nested callbacks, reducing complexity.
--------------

In [9]:
# Build group chat with agent-based orchestrator
workflow = GroupChatBuilder(
    participants=[researcher, writer],
    # Set a hard termination condition: stop after 4 assistant messages
    # The agent orchestrator will intelligently decide when to end before this limit but just in case
    termination_condition=lambda messages: sum(1 for msg in messages if msg.role == "assistant") >= 4,
    orchestrator_agent=orchestrator_agent,
).build()

In [10]:
from typing import cast
from agent_framework import AgentResponseUpdate, Role

task = "What are the key benefits of async/await in Python?"

print(f"Task: {task}\n")
print("=" * 80)

final_conversation: list[Message] = []
last_executor_id: str | None = None

# Run the workflow
async for event in workflow.run(task, stream=True):
    if event.type == "output" and isinstance(event.data, AgentResponseUpdate):
        # Print streaming agent updates
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"[{eid}]:", end=" ", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif event.type == "output":
        # Workflow completed - data is a list of Message
        final_conversation = cast(list[Message], event.data)

if final_conversation:
    print("\n\n" + "=" * 80)
    print("Final Conversation:")
    for msg in final_conversation:
        author = getattr(msg, "author_name", "Unknown")
        text = getattr(msg, "text", str(msg))
        print(f"\n[{author}]\n{text}")
        print("-" * 80)

print("\nWorkflow completed.")

Task: What are the key benefits of async/await in Python?



Final Conversation:

[None]
What are the key benefits of async/await in Python?
--------------------------------------------------------------------------------

[Researcher]
1. **Improved Readability**: Async/await syntax makes asynchronous code look like synchronous code, enhancing readability.

2. **Concurrency**: Allows running I/O-bound operations concurrently without blocking, improving efficiency.

3. **Error Handling**: Simplifies error handling with try/except blocks, similar to synchronous code.

4. **Resource Management**: Efficiently manages resources by pausing function operations until they are needed.

5. **Simplified Code**: Reduces the complexity associated with callback functions and the need for explicit event loops.
--------------------------------------------------------------------------------

[Writer]
6. **Performance**: Increases overall application performance for I/O-bound and high-level structured 